# Data Mining to Discover Fraud Using LSA and LDA

In [290]:
import nltk
import pandas as pd
%pylab inline
from nltk.corpus import stopwords
import gensim
from gensim import corpora, models, similarities
import re

Populating the interactive namespace from numpy and matplotlib


In [1]:
#Load Consumer Financial Protection Bureau public data set.  Found at:
#http://www.consumerfinance.gov/data-research/consumer-complaints/

In [289]:
cfpb = pd.read_csv("../data/Consumer_Complaints_with_Consumer_Complaint_Narratives.csv")

In [295]:
#Its 73k documents describing diffrent types of fraud.  
len(cfpb)

73663

In [291]:
#to delate
#documents = list(cfpb['Consumer complaint narrative'][:1000])

In [962]:
documents = list(cfpb['Consumer complaint narrative'])

In [963]:
# remove common words and tokenize
#Conditions are word is greater than 3 characters, exclude redacted words, 
#and exclude words beginning with a dollar sign. 

texts = [[word.lower() for word in re.split('\.|\,|\n| ',document) if word.lower() not in stopwords.words('english') and 'xx' not in word.lower() and len(word)>3 and '$' not in word]
        for document in documentsall]

/Users/Tianqi/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:3: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  app.launch_new_instance()


In [964]:
# remove words that appear only once
from collections import defaultdict
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1

In [967]:
#breaks each document into a list of strings where each entery in the list is
#string.  Only keeps words that appear more than 10 times in the whole corpus
#of text

texts = [[token for token in text if frequency[token] > 10]
        for text in texts]

In [1083]:
print texts[0]

['party', 'involve', 'equifax', 'please', 'update', 'cell', 'phone', 'number', 'fraud', 'dear', 'madam', 'sole', 'purpose', 'letter', 'inform', 'victim', 'identity', 'theft', 'requesting', 'block', 'following', 'fraudulent', 'information', 'credit', 'reports', 'proceed', 'accordingly', 'file', 'want', 'emphasize', 'none', 'information', 'stated', 'relates', 'transaction', 'made', 'items', 'listed', 'clearly', 'fraudulent', 'therefore', 'requesting', 'deleted', '/blocked', 'name', 'source', 'addition', 'must', 'block', 'fraudulent', 'information', 'victim', 'must', 'identified', 'within', 'business', 'days', 'accepting', 'his/her', 'identity', 'theft', 'report', 'accepts', 'identity', 'theft', 'report', 'also', 'must', 'notify', 'furnishers', 'fraudulent', 'information', 'victim', 'filed', 'report', 'report', 'states', 'information', 'furnished', 'resulted', 'identity', 'theft', 'blocking', 'information', 'furnished', 'fcra', '605b', '1681c-2', 'blocking', 'information', 'credit', 'repo

In [1445]:
cfpb['Consumer complaint narrative'][0]

"To All party involve Equifax XXXX. XXXX XXXX XXXX XXXX, XXXX XXXX XXXX XXXX. XXXX XXXX XXXX, XXXX XXXX XXXX, XXXX. XXXX XXXX XXXX, XXXX XXXX XXXX. \n\nPLEASE UPDATE MY CELL PHONE NUMBER TO MY FRAUD ALLERT XXXX XXXX, XXXX XXXX SOC SEC # XXXX DOBXX/XX/XXXX XXXX XXXX XXXX XXXX, XXXX XXXX Dear Sir or Madam : The sole purpose of this letter is to inform you I have been a victim of identity theft. I am further requesting that you block the following fraudulent information from my credit reports so that I can proceed accordingly with my file. Once more, I want to emphasize none of the information stated below relates to any transaction that I have made. \n\nXXXX, XXXX, XXXX, XXXX XXXX XXXX XXXX ACC # XXXX XXXX ACC # XXXX The items listed are clearly fraudulent ; therefore, I am requesting that they are deleted /blocked by name of source. In addition, A CRA must block the fraudulent information and the victim must be identified within XXXX business days after accepting his/her Identity Theft 

### Create a list of firms mentioned in the Corpus

In [1291]:
companylist = list(cfpb['Company'].unique())

In [1292]:
#generate a list of firms from the CFPB labeled dataset
#drop common firm name endings (ex llc)

companylist = ['_'.join(filter(lambda x: x!='', re.split('\,|\n|\&|Inc|LLC|Co\.|LLP|L.P.|P.C.|\.|Company| ', company))).lower() for company in companylist]

In [1293]:
#2386 firms
len(companylist)

2386

# Phrases Detection

In [1006]:
#generate bigrams for words used adjacent to each other at least twenty times
bigram = gensim.models.Phrases(texts, threshold=20.0)
phrases_texts = bigram[texts]

In [1084]:
print phrases_texts[0]

[u'party_involve', u'equifax', u'please', u'update', u'cell_phone', u'number', u'fraud', u'dear_madam', u'sole_purpose', u'letter', u'inform', u'victim_identity', u'theft', u'requesting', u'block', u'following', u'fraudulent', u'information', u'credit', u'reports', u'proceed_accordingly', u'file', u'want_emphasize', u'none', u'information', u'stated', u'relates_transaction', u'made', u'items', u'listed', u'clearly', u'fraudulent', u'therefore', u'requesting', u'deleted_/blocked', u'name', u'source', u'addition', u'must', u'block', u'fraudulent', u'information', u'victim', u'must', u'identified', u'within', u'business', u'days', u'accepting_his/her', u'identity_theft', u'report', u'accepts_identity', u'theft', u'report', u'also', u'must', u'notify_furnishers', u'fraudulent', u'information', u'victim', u'filed', u'report', u'report', u'states', u'information', u'furnished_resulted', u'identity_theft', u'blocking', u'information', u'furnished_fcra', u'605b_1681c-2', u'blocking', u'informa

In [1031]:
#Create dictionary of vocab
dictionary = corpora.Dictionary(phrases_texts)

In [1521]:
# store the dictionary, for future reference
dictionary.save('phrases_texts.dict')

In [1527]:
# load the dictionary
# dictionary = corpora.Dictionary.load('phrases_texts.dict')

In [1522]:
print(dictionary)

Dictionary(16629 unique tokens: [u'compliant_rules', u'raining', u'honorable_judge', u'00}!!', u'cussed']...)


In [1302]:
# dictionary.token2id.viewkeys()

# word2vec

In [510]:
#These are the paramters of the word2vec model used.  They are the defaults.

# gensim.models.word2vec.Word2Vec(sentences=None, 
# size=100, alpha=0.025, window=5, min_count=5, max_vocab_size=None, 
# sample=0.001, seed=1, workers=3, min_alpha=0.0001, sg=0, hs=0, 
# negative=5, cbow_mean=1, hashfxn=<built-in function hash>, iter=5, 
# null_word=0, trim_rule=None, sorted_vocab=1, batch_words=10000)

In [1056]:
# train word2vec
model = gensim.models.word2vec.Word2Vec(phrases_texts, size=100, sg=1, window=5, workers=2)

In [1523]:
# save the model
model.save('word2vec_model')

In [1524]:
#load the model
# model = gensim.models.Word2Vec.load('word2vec_model')

In [1298]:
wordlist = list(model.index2word)

In [1299]:
len(wordlist)

16237

In [1301]:
print wordlist

[u'credit', u'account', u'would', u'loan', u'payment', u'told', u'bank', u'debt', u'received', u'report', u'information', u'mortgage', u'called', u'company', u'time', u'card', u'never', u'payments', u'call', u'sent', u'paid', u'letter', u'said', u'back', u'could', u'also', u'number', u'made', u'phone', u'money', u'amount', u'since', u'asked', u'home', u'still', u'years', u'balance', u'days', u'make', u'check', u'months', u'contacted', u'date', u'late', u'month', u'reporting', u'name', u'even', u'2015', u'know', u'modification', u'another', u'contact', u'calls', u'stated', u'request', u'accounts', u'file', u'help', u'collection', u'dispute', u'want', u'like', u'need', u'interest', u'send', u'bill', u'address', u'requested', u'last', u'complaint', u'year', u'first', u'went', u'going', u'work', u'documents', u'charge', u'chase', u'process', u'wells_fargo', u'full', u'provide', u'statement', u'however', u'take', u'removed', u'charges', u'reported', u'able', u'closed', u'stating', u'well', 

In [1526]:
# apply

#The result is the 'most similar' word as measured by the cosine similarity
#Here we have crimes that are not fraud as an example
model.most_similar(positive=['federal_crime'], negative=['fruad'], topn=30)

[(u'robbed', 0.32332438230514526),
 (u'think', 0.2907092571258545),
 (u'perpetrator', 0.28019481897354126),
 (u'stupid', 0.27235156297683716),
 (u'stole_identity', 0.26380473375320435),
 (u'anybody', 0.25578364729881287),
 (u'whats', 0.2487524300813675),
 (u'wallet_stolen', 0.24859535694122314),
 (u'maybe', 0.24779406189918518),
 (u'idiot', 0.24674516916275024),
 (u'fool', 0.24637192487716675),
 (u'guys', 0.2452416568994522),
 (u'society', 0.2395319640636444),
 (u'liar', 0.23681335151195526),
 (u'somebody', 0.23668518662452698),
 (u'everyone_else', 0.2365233302116394),
 (u'discriminated', 0.23590087890625),
 (u'that?', 0.2349446415901184),
 (u'people', 0.23451286554336548),
 (u'acted_like', 0.2335684597492218),
 (u'thieves', 0.23102834820747375),
 (u'shame', 0.22961339354515076),
 (u'ripped', 0.22944104671478271),
 (u'cars', 0.2292969524860382),
 (u'educated', 0.22895029187202454),
 (u'purse_stolen', 0.2256564348936081),
 (u'criminals', 0.224974125623703),
 (u'hurts', 0.224455788731575

In [1328]:
#compare two words to see how similar they are
model.similarity('fraud', 'chase')

0.33668147313828539

In [515]:
# model.doesnt_match("breakfast cereal dinner lunch".split())
# model.similarity('woman', 'man')
# model['credit']

#### Search by fraud type to identify firms

In [1317]:
amount = 0 
words = [str(j) for j in wordlist]
newcompanylist = []
for i in companylist:
    if i in words:
        newcompanylist.append(i)
        amount += 1
amount

100

In [1318]:
print newcompanylist

['equifax', 'wells_fargo', 'jpmorgan_chase', 'ally_financial', 'ocwen', 'moneygram', 'experian', 'citibank', 'amex', 'diversified_consultants', 'discover', 'freedom_mortgage', 'aargon_agency', 'bbva_compass', 'genesis_lending', 'aqua_finance', 'delbert_services', 'allied_interstate', 'aldridge_pite', 'afni', 'pressler_pressler', 'statebridge', 'everbank', 'clarity_services', 'grant_weber', 'conserve', 'quicken_loans', 'lexisnexis', 'drivetime', 'americollect', 'seterus', 'transworld_systems', 'mariner_finance', 'comerica', 'hunter_warfield', 'morgan_stanley', 'zwicker_associates', 'cashcall', 'windham_professionals', 'amerisave', 'stellar_recovery', 'nelnet', 'carmax', 'continental_finance', 'williams_fudge', 'mohela', 'selene_finance', 'optimum_outcomes', 'forster_garbus', 'google', 'loandepot', 'provident_funding', 'vhda', 'admin_recovery', 'caine_weiner', 'resolve_partners', 'capio_partners', 'supreme_lending', 'procollect', 'harvard_collections', 'springer_collections', 'great_lake

In [1335]:
imputword = 'identity_theft'
pairlist = []
for company in newcompanylist:
    similarity = model.similarity(imputword, company)
    pair = (company, similarity)
    pairlist.append(pair)

In [1336]:
sorted(pairlist, key=lambda item: -item[1])

[("dyck-o'neal", 0.49189310485012372),
 ('grant_weber', 0.45773354159455104),
 ('revenue_enterprises', 0.44480454123296759),
 ('equifax', 0.43743888942261383),
 ('corelogic', 0.42801920600316212),
 ('lexisnexis', 0.42576461702210244),
 ('capio_partners', 0.42105730512293515),
 ('escallate', 0.42003888169946757),
 ('experian', 0.41923781524454951),
 ('clarity_services', 0.41372059073797085),
 ('continental_finance', 0.41167337372544321),
 ('cohen_associates', 0.40383541825620223),
 ('lendingtree', 0.40257318143829762),
 ('aargon_agency', 0.39311890575407221),
 ('afni', 0.39243623558195778),
 ('pmab', 0.3903938801463902),
 ('pressler_pressler', 0.37324602943188923),
 ('helm_associates', 0.37199554227925707),
 ('contract_callers', 0.36228903554512232),
 ('renton_collections', 0.36008808090753347),
 ('receivable_solutions', 0.35956274009995237),
 ('diversified_consultants', 0.35932921283959385),
 ('peak_acceptance', 0.35690361766699885),
 ('resolve_partners', 0.35593583816470742),
 ('rose_

# Vector Transformations to compare documents

In [1067]:
# add unique integer id for words in text
dictionary = corpora.Dictionary(phrases_texts)

In [1068]:
# The function doc2bow() simply counts the number of occurences of each distinct word,
#converts the word to its integer word id 
#and returns the result as a sparse vector.
corpus = [dictionary.doc2bow(text) for text in phrases_texts]

In [1528]:
# store to disk, for later use
corpora.MmCorpus.serialize('original_corpus.mm', corpus)  

In [1529]:
# load the corpus
# corpus = corpora.MmCorpus('original_corpus.mm')

In [1451]:
print corpus[0]

[(0, 2), (1, 2), (2, 1), (3, 1), (4, 2), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 12), (18, 1), (19, 1), (20, 1), (21, 2), (22, 2), (23, 1), (24, 1), (25, 1), (26, 1), (27, 2), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 7), (37, 2), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 5), (47, 1), (48, 1), (49, 2), (50, 3), (51, 1), (52, 1), (53, 1), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 3), (63, 1), (64, 1), (65, 4), (66, 1), (67, 1), (68, 3), (69, 1)]


In [1452]:
dictionary.get(36)

u'credit'

In [1071]:
# step 1 -- initialize a model
# normalizes the vector occuranice by taking into account its freqency in
#both the document as well as the corpus.

tfidf = models.TfidfModel(corpus)

In [1072]:
# step 2 -- use the model to transform vectors
corpus_tfidf = tfidf[corpus]

In [1530]:
# store to disk, for later use
corpora.MmCorpus.serialize('corpus_tfidf.mm', corpus)  

In [1531]:
# load the corpus
# corpus_tfidf = corpora.MmCorpus('corpus_tfidf.mm')

In [1073]:
print corpus_tfidf[0]

[(0, 0.14929067223107642), (1, 0.14158864040774302), (2, 0.038425330138476294), (3, 0.14224590187423578), (4, 0.28449180374847155), (5, 0.05376890089063228), (6, 0.029784639543662776), (7, 0.08471186504757271), (8, 0.05522923501796895), (9, 0.14224590187423578), (10, 0.07610403504357215), (11, 0.03777214884591696), (12, 0.14224590187423578), (13, 0.03558537986699427), (14, 0.048242637710893044), (15, 0.06047689071350299), (16, 0.148564895429346), (17, 0.27696265391529457), (18, 0.056117859902628894), (19, 0.0881308710009035), (20, 0.060320623183916795), (21, 0.1040096555394597), (22, 0.08220679452870829), (23, 0.14224590187423578), (24, 0.14224590187423578), (25, 0.14449783344831896), (26, 0.14574762257358861), (27, 0.13190830811162801), (28, 0.037054494551263376), (29, 0.050724709805395035), (30, 0.14224590187423578), (31, 0.05378556261003734), (32, 0.14224590187423578), (33, 0.059006050453735505), (34, 0.0425665090958153), (35, 0.11168115701846636), (36, 0.08598740681155351), (37, 0.

# Latent Semantic Indexing, LSI (or LSA)

In [1075]:
# initialize an LSI transformation
lsi = models.lsimodel.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=500) 

In [1532]:
# save the lsi model
lsi.save('lsi_model.lsi') 

In [1533]:
# load the model
# lsi = models.LsiModel.load('lsi_model.lsi')

In [1250]:
#Top 5 Weighted LSI Topics
lsi.print_topics(5,  num_words=5)

[(0,
  u'0.185*"account" + 0.178*"credit" + 0.154*"payment" + 0.150*"debt" + 0.149*"loan"'),
 (1,
  u'0.327*"report" + 0.266*"credit" + 0.238*"debt" + -0.236*"loan" + -0.208*"mortgage"'),
 (2,
  u'-0.441*"debt" + -0.232*"call" + -0.194*"number" + -0.187*"calls" + -0.176*"phone"'),
 (3,
  u'0.396*"card" + -0.281*"loan" + 0.262*"account" + -0.250*"mortgage" + -0.210*"modification"'),
 (4,
  u'-0.416*"debt" + -0.241*"paid" + -0.239*"payment" + 0.195*"experian" + 0.189*"number"')]

In [1077]:
# create a double wrapper over the original corpus: bow->tfidf->fold-in-lsi
corpus_lsi = lsi[corpus_tfidf]

In [1081]:
#topic one is the most similar for the first document.
sorted(corpus_lsi[0], key=lambda item: -item[1])[:5]

[(1, 0.17866985139890593),
 (36, 0.15349369821976436),
 (0, 0.1478655446760373),
 (45, 0.11335023122819589),
 (33, 0.10841507612070858)]

In [1082]:
#the words that make up topic 1
lsi.show_topic(1, topn=5)

[(u'report', 0.32741963949960445),
 (u'credit', 0.26554459008704512),
 (u'debt', 0.23849268977815119),
 (u'loan', -0.23598558784129159),
 (u'mortgage', -0.20836277962838556)]

# Similarity Queries

In [1087]:
# transform corpus to LSI space and index it
# selection of 500 feature space was arbitrary 
index_lsi = similarities.docsim.MatrixSimilarity(corpus_lsi, num_features=500)

In [1465]:
# input the doc to be compared
# here a 'fake tweet' was made up
#doc = cfpb['Consumer complaint narrative'][120]

doc = "my bank is a real bummer.  They call me all the time.  My credit card is maxed out.  I just don't know what to do with my life.  Ugg"

In [1466]:
# transform the input doc to match its format with the corpus
doc = [word.lower() for word in re.split('\.|\,|\n| ',doc) if word.lower() not in stopwords.words('english') and 'xx' not in word.lower() and len(word)>3 and '$' not in word]   
doc = [token for token in doc if frequency[token] > 10]
vec_bow = dictionary.doc2bow(bigram[doc])
vec_lsi = lsi[tfidf[vec_bow]] # convert the query to LSI space

In [1467]:
# perform a similarity query against the corpus
sims_lsi = index_lsi[vec_lsi] 

In [1469]:
sims_lsi_sorted = sorted(enumerate(sims_lsi), key=lambda item: -item[1])

In [1470]:
#document 4671 has the highest cosine similarity
sims_lsi_sorted[:50]

[(4671, 0.55112529),
 (51536, 0.51052439),
 (15084, 0.47676924),
 (17314, 0.47676924),
 (40536, 0.46828932),
 (16364, 0.46624014),
 (50001, 0.46215898),
 (45879, 0.46073705),
 (39065, 0.45395714),
 (19251, 0.45042437),
 (24705, 0.44922185),
 (14182, 0.44806379),
 (69644, 0.44606543),
 (17601, 0.44385242),
 (40075, 0.44311649),
 (69141, 0.43823844),
 (30877, 0.43340981),
 (62572, 0.43104094),
 (28823, 0.42903489),
 (73457, 0.42867807),
 (38766, 0.42135194),
 (25782, 0.41855085),
 (2793, 0.41850418),
 (10803, 0.41758773),
 (17702, 0.41703987),
 (4021, 0.41680861),
 (54751, 0.41619033),
 (27269, 0.41316092),
 (41001, 0.41065508),
 (27137, 0.40960854),
 (28399, 0.40466022),
 (988, 0.40380347),
 (37647, 0.40177476),
 (1603, 0.40085268),
 (10559, 0.40055239),
 (73211, 0.40012681),
 (61353, 0.39847565),
 (23642, 0.39819118),
 (65376, 0.39748532),
 (59781, 0.39708734),
 (67918, 0.39611313),
 (36303, 0.39536905),
 (56280, 0.39525348),
 (53703, 0.39352307),
 (64803, 0.39352307),
 (22631, 0.39329

In [1472]:
cfpb['Consumer complaint narrative'][4671]

"I call the credit card company and they was not going to help me. I do know what they are taking about I did n't got a credit card from know one please help me with this and thank you \n"

In [1535]:
cfpb['Consumer complaint narrative'][51536]

"My credit card from PNC Bank Credit Card was compromised when XXXX had a security breach ... I was notified from PNC Bank that they would be sending me a new credit card in XX/XX/XXXX, to replace the old credit card that may have been compromised from the target breach at the end of XXXX XXXX ... I also spoke to them at this time about making a payment and they told me to hold off until I received the new credit card ... .I was supposed to receive the new card in XX/XX/XXXX, however, I did n't receive the new card until XX/XX/XXXX, almost a month and a half later ... .Once I received the new card I immediately called PNC Bank to activate the new card to replace the older one ... I ask the representative at the time if the account was open and activated and he stated that it was, however, at the time I talked to this representative, it had already been closed by PNC Bank ... The rep told me to set up a payment in the next week or so since the account was running behind due to PNC Bank 

Both complaints were dealing with what the tweet was roughly about.  Maxed out credit cards and too many calls

# Latent Dirichlet Allocation, LDA

LDA assumes documents are produced from a mixture of topics. Those topics then generate words based on their probability distribution, like the ones in our walkthrough model. In other words, LDA assumes a document is made from the following steps:

    Determine the number of words in a document. Let’s say our document has 6 words.

    Determine the mixture of topics in that document. For example, the document might contain 1/2 the topic “health” and 1/2 the topic “vegetables.”

    Using each topic’s multinomial distribution, output words to fill the document’s word slots. In our example, the “health” topic is 1/2 our document, or 3 words. The “health” topic might have the word “diet” at 20% probability or “exercise” at 15%, so it will fill the document word slots based on those probabilities.

Given this assumption of how documents are created, LDA backtracks and tries to figure out what topics would create those documents in the first place.

num_topics: required. An LDA model requires the user to determine how many topics should be generated. Our document set is small, so we’re only asking for three topics.

id2word: required. The LdaModel class requires our previous dictionary to map ids to strings.

passes: optional. The number of laps the model will take through corpus. The greater the number of passes, the more accurate the model will be. A lot of passes can be slow on a very large corpus.


In [1096]:
lda = gensim.models.ldamodel.LdaModel(corpus_tfidf, id2word=dictionary, num_topics=500, passes=5)

In [1542]:
# save the lda model
lda.save('lda_model.lda') 

In [1536]:
# load the model
# lda = models.LdaModel.load('lda_model.lda')

In [1097]:
#lda weighted topics
lda.show_topics(num_topics=5, num_words=10, log=False, formatted=True)

[(323,
  u'0.492*wells_fargo + 0.088*every_month + 0.027*mortgage + 0.020*earn + 0.019*routing_number + 0.014*bank + 0.012*charged + 0.011*interest + 0.011*payment + 0.010*month'),
 (185,
  u'0.136*outstanding_balance + 0.133*greentree + 0.099*affected + 0.084*changing + 0.071*occur + 0.066*terms_conditions + 0.059*language + 0.012*free_night + 0.010*agreement + 0.009*statement'),
 (417,
  u'0.156*limitations + 0.121*although + 0.104*otherwise + 0.094*attempted + 0.077*past_statute + 0.059*advertising + 0.048*deferred_interest + 0.047*fdic + 0.019*balance! + 0.019*collect'),
 (216,
  u'0.129*apartment + 0.059*ended + 0.055*class_action + 0.042*satisfy + 0.029*calculate + 0.026*recast + 0.020*lease + 0.019*navy + 0.017*suit + 0.014*in-house'),
 (358,
  u'0.023*self + 0.012*birth + 0.011*valued + 0.010*bank + 0.010*classes + 0.010*weekends + 0.008*scenario + 0.008*bbva_compass + 0.008*waste + 0.008*would')]

In [1098]:
result = lda.do_estep(corpus_tfidf, state=None)

In [1540]:
# result[0]

In [1104]:
# get topic from a document
lda.get_document_topics(corpus_tfidf[0], minimum_probability=None)

[(67, 0.84665391707723259), (347, 0.011119764244878939)]

In [1102]:
corpus_lda = lda[corpus_tfidf]

#### find topic

In [1457]:
#the topics of document 120.  In this case its strongly topic 70 based
#on cosine similarity
corpus_lda[120]

[(70, 0.60316808428124757),
 (196, 0.024317451034799365),
 (228, 0.037564679327827102),
 (279, 0.0306313631290961),
 (311, 0.010035686876650801),
 (376, 0.051048644739457659),
 (483, 0.010813579719624086),
 (487, 0.029608528246048788)]

In [1458]:
#document 120
cfpb['Consumer complaint narrative'][120]

"Macy 's has charge me 8 month interest on a closed account that i request to be closed on XXXX XXXX 2011 because high interest rate it does n't state after you closed an account that Macy 's is still able to charger you interest it only state that Macy 's can collect the remain balance which i owe. The agreement that i sign was terminated upon my request if it is closed you are only to collect the the remain balance Macy 's agreement was terminated so were the late fee Macy 's has not sent me any type of information that state after account is closed Macy 's is able to charger you late fee. so i decide to file a complaint against Macy 's so please investigate this case Thank you Sincerely XXXX XXXX XXXX \n"

In [1460]:
# show the topic of topic 70
lda.show_topic(70, topn=20)

[(u'macy', 0.24287284558174288),
 (u'section', 0.098466706640208582),
 (u'compliance', 0.063725650457652203),
 (u'terminated', 0.058676242281686525),
 (u'statement', 0.025162460329541454),
 (u'card', 0.024593688750382425),
 (u'fraud', 0.02319532474420127),
 (u'charges', 0.020496763554601807),
 (u'install', 0.015645977436904917),
 (u'late', 0.01523082880758332),
 (u'ride', 0.014387913918256475),
 (u'account', 0.012608135489035027),
 (u'white', 0.011291042611309384),
 (u'credit', 0.0094080941082986275),
 (u'closed', 0.0092118595143336948),
 (u'consumer', 0.0086652717518981934),
 (u'statements', 0.0082121679305062721),
 (u'balance', 0.0079269195388321657),
 (u'interest', 0.0074107438382308407),
 (u'copy', 0.0069275414683960422)]

### LDA Similarities

In [1107]:
#create lda index
index_lda = similarities.MatrixSimilarity(corpus_lda, num_features=500)

In [1146]:
doc = cfpb['Consumer complaint narrative'][120]
# transform the input doc to match format of corpus
doc = [word.lower() for word in re.split('\.|\,|\n| ',doc) if word.lower() not in stopwords.words('english') and 'xx' not in word.lower() and len(word)>3 and '$' not in word]   
doc = [token for token in doc if frequency[token] > 10]
vec_bow = dictionary.doc2bow(bigram[doc])
vec_lda = lda[tfidf[vec_bow]] # convert the query to LSI space

In [1147]:
sims_lda = index_lda[vec_lda] 

In [1148]:
sims_lda_sorted = sorted(enumerate(sims_lda), key=lambda item: -item[1])

In [1149]:
#documents with the their similarity consines
#NOTE these results show overfitting.
sims_lda_sorted[:50]

[(120, 0.99968714),
 (121, 0.97623652),
 (464, 0.95726317),
 (45045, 0.88136822),
 (58122, 0.85405421),
 (11545, 0.81315798),
 (36130, 0.80075783),
 (54293, 0.78365052),
 (37713, 0.76949638),
 (16421, 0.76732326),
 (15094, 0.75462371),
 (69443, 0.7489934),
 (15778, 0.74737781),
 (35800, 0.74461526),
 (24138, 0.74208772),
 (56326, 0.73926359),
 (16567, 0.7377578),
 (14150, 0.73404068),
 (26351, 0.73151904),
 (53976, 0.72465324),
 (30135, 0.72377181),
 (39292, 0.72368532),
 (61705, 0.72174042),
 (34957, 0.7186166),
 (16677, 0.71693724),
 (13747, 0.71445197),
 (45527, 0.71005762),
 (9198, 0.70990914),
 (7506, 0.70902747),
 (54251, 0.70898944),
 (8057, 0.7005468),
 (33558, 0.69586003),
 (27266, 0.69248652),
 (42092, 0.69086099),
 (55418, 0.69066769),
 (23160, 0.69003928),
 (5159, 0.68668437),
 (69799, 0.68633997),
 (13116, 0.6854614),
 (27914, 0.68541598),
 (18340, 0.68307656),
 (42422, 0.68204719),
 (13872, 0.67977673),
 (27708, 0.67634875),
 (64260, 0.67627621),
 (66774, 0.66595614),
 (1

In [1462]:
cfpb['Consumer complaint narrative'][120]

"Macy 's has charge me 8 month interest on a closed account that i request to be closed on XXXX XXXX 2011 because high interest rate it does n't state after you closed an account that Macy 's is still able to charger you interest it only state that Macy 's can collect the remain balance which i owe. The agreement that i sign was terminated upon my request if it is closed you are only to collect the the remain balance Macy 's agreement was terminated so were the late fee Macy 's has not sent me any type of information that state after account is closed Macy 's is able to charger you late fee. so i decide to file a complaint against Macy 's so please investigate this case Thank you Sincerely XXXX XXXX XXXX \n"

In [1546]:
cfpb['Consumer complaint narrative'][121]

"In XX/XX/2014 I purchased an Solar Air Cond. system. I have found out that the system installed is NOT an Solar Air Cond. system and that the outside condensing unit and inside air handler should have NEVER been connected to an Solar panel per the manufacture. \nI have placed criminal charges against the company's owner. The company XXXX that sold me the system arranged for financing thru Aqua Finance Inc. I have explained several times over the past several months that XXXX DID NOT install a Solar system they installed a plane old A/C and air handler system that is connected to a Solar panel that should have never been done. Now Aqua is coming after me for non payment and has placed past due notices on my credit record. I'm NOT going to pay for something that is a FRAUD. Please help me with getting ride of this debit that Ive tried my very best to let EVERYONE know that I didn't get what I paid for."

The results do not pass the human test.  This indicates the LDA model was overfit.

# Compare LSA with LDA

In [1340]:
def test(docnumber):
    # select an input
    doc = cfpb['Consumer complaint narrative'][docnumber]
    # transform the input doc
    doc = [word.lower() for word in re.split('\.|\,|\n| ',doc) if word.lower() not in stopwords.words('english') and 'xx' not in word.lower() and len(word)>3 and '$' not in word]   
    doc = [token for token in doc if frequency[token] > 10]
    vec_bow = dictionary.doc2bow(bigram[doc])
    
    vec_lsi = lsi[tfidf[vec_bow]]
    vec_lda = lda[tfidf[vec_bow]] 
    
    sims_lsi = index_lsi[vec_lsi] 
    sims_lda = index_lda[vec_lda] 
    
    sims_lsi_sorted = sorted(enumerate(sims_lsi), key=lambda item: -item[1])
    sims_lda_sorted = sorted(enumerate(sims_lda), key=lambda item: -item[1])
    
    a = 0
    b = 20
    for i in sims_lsi_sorted[:b]:
        for j in sims_lda_sorted[:b]:
            if i[0]==j[0]:
                a+=1
    print docnumber, '| In top similiarty', b , "| identical amount =", a

In [1341]:
for i in range(100):
    test(i)

0 | In top similiarty 20 | identical amount = 14
1 | In top similiarty 20 | identical amount = 1
2 | In top similiarty 20 | identical amount = 2
3 | In top similiarty 20 | identical amount = 1
4 | In top similiarty 20 | identical amount = 1
5 | In top similiarty 20 | identical amount = 5
6 | In top similiarty 20 | identical amount = 1
7 | In top similiarty 20 | identical amount = 1
8 | In top similiarty 20 | identical amount = 1
9 | In top similiarty 20 | identical amount = 1
10 | In top similiarty 20 | identical amount = 1
11 | In top similiarty 20 | identical amount = 3
12 | In top similiarty 20 | identical amount = 1
13 | In top similiarty 20 | identical amount = 1
14 | In top similiarty 20 | identical amount = 1
15 | In top similiarty 20 | identical amount = 1
16 | In top similiarty 20 | identical amount = 5
17 | In top similiarty 20 | identical amount = 1
18 | In top similiarty 20 | identical amount = 1
19 | In top similiarty 20 | identical amount = 1
20 | In top similiarty 20 | i

# Test with Tweets

### These tweets were unrelated to the topic of the project and are therefore not that informative but they do prove the methodology will work with a stream of all tweets from New Yorkers


In [1371]:
twitterdata = pd.read_csv("../data/csv_tweets_v1.csv", header=None)

In [1372]:
tweets = list(twitterdata.iloc[:,0])

In [1374]:
len(tweets)

1000

In [1398]:
length = []
for i in tweets:
    if type(i) == str:
        length.append(len(i.split()))

In [1404]:
average(length)

10.382765531062125

In [1425]:
amount = 0
for j,i in enumerate(tweets):
    if type(i) == str:
        if 'uber' in i.split():
            amount += 1
            print j
            print i, '\n'

2
When I took the uber to Jersey during rush hour instead of the train &lt;&lt;. Top 5 worst decisions my bill was dirt 

16
Meu pai esqueceu a chave e o que eu fiz? Chamei uber e ele entregou kkkkkkk 

89
RT @TheGabbieShow: this uber driver just told me a 7 minute story of one time she tried to park in a tight spot. she should start a youtube 

101
uber for fuse 

125
Like nah I'll get a uber to bring me back to my car tomorrow 

159
When your uber driver is also a delivery driver for pizzeria 260 &amp; knows who you are ... 

183
@JoeyRossi67 prolly hopefully if i uber 

192
@HossanLeong Judging by the way uber is heading, we wont even need them drivers even. Hohoho 

197
My uber driver looks like @tomhanks 

227
RT @robertmoreactor: uno va a robar gallinas tranquilo y aparece el uber de los locutores.
pero don @RodrigoRomano76 ya pago las cuotas. ht 

249
uber everywhere.. in the back of that bih with a ruger 

255
RT @AndreaRussett: one time I was leaving a party and thought I saw 

In [1426]:
amount

46

In [1428]:
tweets[435]

'how are so many uber drivers so bad at driving'

In [1429]:
tweets[838]

'Will forever be grateful to the uber driver that gave me 10 bucks for school '

In [1483]:
def findtopic(text):
    # select an input
    doc = text
    # transform the input doc
    doc = [word.lower() for word in re.split('\.|\,|\n| ',doc) if word.lower() not in stopwords.words('english') and 'xx' not in word.lower() and len(word)>3 and '$' not in word]   
    doc = [token for token in doc if frequency[token] > 10]
    vec_bow = dictionary.doc2bow(bigram[doc])
    
    lsi_result = sorted(lsi[tfidf[vec_bow]], key=lambda item: -item[1])[:5]
    
    lda_result = sorted(lda[tfidf[vec_bow]], key=lambda item: -item[1])[:5]
     
    return lsi_result, lda_result

In [1442]:
lsi_result, lda_result = findtopic(tweets[435])

In [1443]:
lsi_result

[(334, 0.055437073112975126),
 (362, 0.049644111856967064),
 (369, 0.039906952711141037),
 (339, 0.034718401299053604),
 (375, 0.031585259193952837)]

In [1444]:
lda_result

[(181, 0.3340790005668855), (473, 0.28662742544664166)]

In [1437]:
tweets[435]

'how are so many uber drivers so bad at driving'

In [1436]:
lsi.show_topic(334, topn=10)

[(u'many', 0.22123507861249536),
 (u'late_fees', 0.17409796859497922),
 (u'program', -0.1600144866663579),
 (u'hold', -0.13854550632451509),
 (u'week', 0.12874516018375781),
 (u'pulled', 0.12703567176990702),
 (u'using', -0.11521778459940649),
 (u'finance', -0.11358310623751222),
 (u'well', -0.11255244921557819),
 (u'father', 0.11156755397144251)]

In [1440]:
lda.show_topic(181, topn=10)

[(u'auto', 0.24282737870550253),
 (u'financing', 0.11003234742644261),
 (u'figure', 0.070307551732642068),
 (u'cycle', 0.053156312757885994),
 (u'actually', 0.044240836117131872),
 (u'driving', 0.043318378979432158),
 (u'loan', 0.028899422801468243),
 (u'driver', 0.017079374471690906),
 (u'finance', 0.01625920604986154),
 (u'vehicle', 0.015448435000089649)]

### Test the thredthod

In [1508]:
def thr_findtopic(text):
    # select an input
    doc = text
    # transform the input doc
    doc = [word.lower() for word in re.split('\.|\,|\n| ',doc) if word.lower() not in stopwords.words('english') and 'xx' not in word.lower() and len(word)>3 and '$' not in word]   
    doc = [token for token in doc if frequency[token] > 10]
    vec_bow = dictionary.doc2bow(bigram[doc])
    
    lsi_result = sorted(lsi[tfidf[vec_bow]], key=lambda item: -item[1])
    
    lda_result = sorted(lda[tfidf[vec_bow]], key=lambda item: -item[1])
     
    return lsi_result, lda_result

In [1509]:
lsi_list = []
lda_list = []
number_of_empty = 0
for j, i in enumerate(tweets):
    try:
        lsi_result, lda_result = thr_findtopic(i)
        if lsi_result == [] or lda_result == []:
            number_of_empty += 1
        else:
            lsi_list.append((j,lsi_result[0][1]))
            lda_list.append((j,lda_result[0][1]))
    except:
        pass

In [1518]:
lsi_list_sorted = sorted(lsi_list, key=lambda item: -item[1])

In [1547]:
for i in lsi_list_sorted:
    print i[1], '|', tweets[i[0]]

0.325653831556 | @josenation1. Work 
0.325653831556 | -when I'm bored at work ... https://t.co/OObnKO9GIZ
0.264847548972 | @ImDontai Today was just the tip of the iceberg :/
0.264097933407 | Bucs PLEASE call Jax
0.25511424998 | Told you @NYPost_Loftis  https://t.co/pu1JNK6kse
0.25511424998 | Told you it was elliott
0.244504470473 | @A_HumbleMan I want a bouquet 
0.244504470473 | I want cheesecake
0.244504470473 | I want to puke https://t.co/v1PpDGWaIB
0.244504470473 | I want to be @carrieffisher
0.244504470473 | God I want Jerry Jones to die.
0.233904503451 | One lane closed in #NewYork on 5th Ave SB between 57th St and 56th St #traffic https://t.co/cLJ9IKJQWV
0.23124049763 | dallas gonna be a problem smh
0.228077517616 | RT @PanteraMexicat: Ya pueden pedir su uber pa q los lleven a la fiesta #ubersecret show. *entrada gratis usuarios https://t.co/xrrlm1B63Z
0.224898258154 | @wsvn @Uber that all these company do is just want money.
0.220387941235 | @karanewman columns/opinion...you kno